In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Define dataset paths
data_dir = "dataset_fer"
train_dir = f"{data_dir}/train"
test_dir = f"{data_dir}/test"